In [1]:
# TODAS LAS LIBRERÍAS PARA SCRAPEAR
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException, NoSuchElementException, WebDriverException, NoSuchWindowException
from selenium.webdriver.common.alert import Alert
import time
import pandas as pd
import shutil
from datetime import datetime
from selenium.webdriver.common.action_chains import ActionChains
import requests
import PyPDF2
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import sys
import os
from dotenv import load_dotenv
from bs4 import BeautifulSoup

# Scraper para variables

## Scrapper cédula profesional

### Conexión al navegador

In [2]:
chrome_options = Options()
driver = webdriver.Chrome(options=chrome_options)
start_url = 'https://cedulaprofesionalsep.online/#Consulta_de_Cedula_Profesional'
driver.get(start_url)
# Modificar el tamaño de la ventana
#driver.execute_script("document.body.style.zoom='50%'")

> Función para obtener el tamaño de una solicitud HTTP

In [3]:
def get_request_size(url):
    response = requests.get(url)
    return len(response.content)

### Leer archivo CSV y obtener lista de nombres

In [4]:
init=2
final=500

df = pd.read_excel('Z:/Data/HSBC/Asignaciones/ASG_TDC_IA_ENERO_JUNIO.xlsx')
df['Nombre'] = df.Nombre.apply(lambda X: X.replace('/', ' ').replace('*', '').rstrip())
df_filtered = df['Nombre']
df_filtered.drop_duplicates(inplace=True)
df_input = pd.DataFrame(df_filtered.reset_index(drop=True)).iloc[init:final]



df_input['NombreSplit'] = df_input.Nombre.apply(lambda X: split_name(X))
df_input['Nombre'] = df_input.NombreSplit.apply(lambda X: X[0])
df_input['Paterno'] = df_input.NombreSplit.apply(lambda X: X[1])
df_input['Materno'] = df_input.NombreSplit.apply(lambda X: X[2])

# Concatenación
df_input['Nombre_Completo'] = df_input.apply(lambda row: ' '.join([row['Nombre'], row['Paterno'], row['Materno']]), axis = 1)
# Cambiar nombre
#df.rename(columns={'Nombre':'Nombre'}, inplace=True)
df_input.tail()

FileNotFoundError: [Errno 2] No such file or directory: 'Z:/Data/HSBC/Asignaciones/ASG_TDC_IA_ENERO_JUNIO.xlsx'

In [ ]:
df_input.head()

### Scrapper

In [ ]:
# Código para modificar la ventana
driver.execute_script("window.scrollTo(10, 7600);")

In [ ]:
data=[]
#total_data_downloaded = 0
for index, row in df_input.iterrows():
    nombre_persona = row['Nombre_Completo']
   
    # Se escrolea hasta la parte específica 
    driver.execute_script("window.scrollTo(10, 7600);")
    time.sleep(5)
    
    # Se ingresan las variables del scrapeo
    nombre=row['Nombre']
    paterno=row['Paterno']
    materno=row['Materno']
    
    # Ingresar nombres
    celda_nombre = driver.find_element(By.XPATH, '//*[@id="input-nombre"]')
    celda_nombre.clear()
    celda_nombre.send_keys(str(nombre))

    #Ingresar Apellido Paterno
    celda_paterno = driver.find_element(By.XPATH, '//*[@id="input-apaterno"]')
    celda_paterno.clear()
    celda_paterno.send_keys(str(paterno))

    # Ingresar Apellido Materno
    celda_materno = driver.find_element(By.XPATH, '//*[@id="input-amaterno"]')
    celda_materno.clear()
    celda_materno.send_keys(str(materno))

    # Click en botón buscar
    buscar = driver.find_element(By.XPATH, '//*[@id="container-form-1"]/form/div[4]/div/input')
    buscar.click()
    time.sleep(2)

    max_wait_time = 5  # Tiempo máximo de espera en segundos
    start_time = time.time()  # Momento de inicio de espera

    while time.time() - start_time < max_wait_time:
        try:
            alert = driver.switch_to.alert
            alert.accept()
            driver.refresh()
            break  # Salir del bucle si se encuentra la ventana emergente
        except:

            time.sleep(1)  # Esperar 1 segundo antes de intentar nuevamente
            
            


    time.sleep(5)
    
    # Esperar a que los elementos estén visibles
    ver_botones = WebDriverWait(driver, 10).until(EC.visibility_of_all_elements_located((By.CLASS_NAME, 'ver-mas')))

    
    for ver in ver_botones:
        nombre_boton = ver.get_attribute("value")
        print(nombre_boton)
        if unidecode(nombre_boton.lower()) == unidecode(nombre_persona.lower()):
            cantidad_registros = len(ver_botones)

            ver.click()
            
            # Encontrar los elementos de entrada de texto por su id
            input_cedula = driver.find_element(By.ID, "input-cedula-result")
            input_tipo = driver.find_element(By.ID, "input-tipo-result")
            input_sexo = driver.find_element(By.ID, "input-sexo-result")
            input_nombre = driver.find_element(By.ID, "input-nombre-result")
            input_paterno = driver.find_element(By.ID, "input-paterno-result")
            input_materno = driver.find_element(By.ID, "input-materno-result")
            input_escuela = driver.find_element(By.ID, "input-escuela-result")
            input_titulo = driver.find_element(By.ID, "input-titulo-result")

            # Obtener los valores de los campos
            valor_cedula = input_cedula.get_attribute("value")
            valor_tipo = input_tipo.get_attribute("value")
            valor_sexo = input_sexo.get_attribute("value")
            valor_nombre = input_nombre.get_attribute("value")
            valor_paterno = input_paterno.get_attribute("value")
            valor_materno = input_materno.get_attribute("value")
            valor_escuela = input_escuela.get_attribute("value")
            valor_titulo = input_titulo.get_attribute("value")

            # Agregar los valores a la lista de diccionarios
            data.append({
            "Cédula": valor_cedula,
            "Tipo de Cédula": valor_tipo,
            "Sexo": valor_sexo,
            "Nombre": valor_nombre,
            "Paterno": valor_paterno,
            "Materno": valor_materno,
            "Escuela": valor_escuela,
            "Título": valor_titulo
            })
            # Encontrar los elementos de entrada de texto por su id
            cerrar = driver.find_element(By.XPATH, '//*[@id="cerrar_resut_personal"]')
            cerrar.click()
       
        
        # Obtener el tamaño de la página descargada
        #page_source_size = len(driver.page_source.encode('utf-8'))
        #total_data_downloaded += page_source_size
        
driver.quit()            
df_data = pd.DataFrame(data)
df_data.to_csv('Z:/Data/HSBC/SCRAPPING/Cédulas/0_500_.csv')
    

                



In [ ]:
df_data.tail(10)

In [ ]:
nombre_buscado = "jose de jesus"
resultado = df_input.loc[df_input['Nombre'] == nombre_buscado]
resultado

In [ ]:
# Convertir los datos en un DataFrame y guardar en CSV              
df_data = pd.DataFrame(data)
df_data.to_csv('Z:/Data/HSBC/SCRAPPING/Cédulas/0_192_.csv')
    

## IFT

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import zipfile
import os
from datetime import datetime
from selenium.webdriver.common.action_chains import ActionChains
import warnings
warnings.filterwarnings('ignore')

def ift_automation():
    # Configura las opciones de Chrome para la descarga
    #download_folder = "/home/bryan/Documentos/airflow/outputs/ift"
    chrome_options = Options()
    #chrome_options.add_argument('--headless')
    chrome_options.add_experimental_option("prefs", {
        "download.prompt_for_download": False,
        #"download.default_directory": download_folder,  # Utiliza la ubicación actual del script como carpeta de descarga
        "download.directory_upgrade": True,
        "safebrowsing.enabled": True, # Desactiva la verificación de seguridad de descargas
        "profile.default_content_settings.popups":0,
        "profile.default_content_setting_values.automatic_downloads": 2,
    })

    # Configuración para ingresar al explorador
    remote_webdriver = 'remote_chromedriver'
    driver = webdriver.Chrome( options=chrome_options)
    url = "https://sns.ift.org.mx:8081/sns-frontend/planes-numeracion/descarga-publica.xhtml"
    driver.get(url)
    wait = WebDriverWait(driver, 10)
    time.sleep(10)
    button = wait.until(EC.element_to_be_clickable((By.ID, "FORM_planes:BTN_planPublico1")))
    button.send_keys(Keys.ENTER)

    # Espera un tiempo para que la descarga se complete
    tiempo_espera = 60  # Puedes ajustar este valor según sea necesario
    tiempo_transcurrido = 0
    archivo_zip = None

    while tiempo_transcurrido < tiempo_espera:
        archivos_en_carpeta = os.listdir("/opt/airflow/outputs/ift/")
        for archivo in archivos_en_carpeta:
            if archivo.endswith('.zip'):
                archivo_zip = archivo
                break

        if archivo_zip:
            break

        time.sleep(1)
        tiempo_transcurrido += 1

    if archivo_zip:
        # Abre el archivo ZIP en modo de lectura
        with zipfile.ZipFile(os.path.join("/opt/airflow/outputs/ift/", archivo_zip), 'r') as zf:
            # Extrae todos los archivos y carpetas dentro del archivo ZIP en la carpeta de destino
            zf.extractall("/opt/airflow/outputs/ift/")

    driver.quit()
ift_automation()

## Scraper Gepard Center

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import click
import os
import shutil
from datetime import datetime



def gepard_automation(userid, passwo):
    url = "https://www.message-center.com.mx/"
    # Configuración para evitar notificaciones
    chrome_options = Options()
    chrome_options.add_experimental_option("prefs", {
    "download.prompt_for_download": False,  # Desactiva la ventana emergente de descarga
    "download.directory_upgrade": True,
    "safebrowsing.enabled": False,  # Desactiva la verificación de seguridad de descargas
})
    # Configuración para ingresar al explorador
    driver = webdriver.Chrome(options = chrome_options)
    driver.get(url)
    wait = WebDriverWait(driver, 10)

    # Configurar credenciales
    user = driver.find_element(By.ID,'Usr')
    user.send_keys(userid)
    password = driver.find_element(By.ID, 'Pass')
    password.send_keys(passwo)
    ingresar = driver.find_element(By.CLASS_NAME, 'cssButtonMn')
    ingresar.click()
    time.sleep(10)

    # Ir a mensajes
    mensajes = driver.find_element(By.XPATH, '//*[@id="MenuPanel"]/table/tbody/tr/td[2]/img')
    mensajes.click()
    time.sleep(5)
    # Agregar datos
    buscar = driver.find_element(By.CLASS_NAME, 'cssBtn')
    buscar.click()
    # Exportar Excel
    export = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'img[title="Exportar a Excel..."]')))
    export.click()
    time.sleep(10)
    driver.quit()


def move_downloaded_file(source_directory, target_directory, file_extension):
    # Enumera los archivos en el directorio de origen
    for filename in os.listdir(source_directory):
        if filename.endswith(file_extension):
            
            # Obtén la fecha actual en el formato deseado (A-M-D-H-M-S)
            current_datetime = datetime.now().strftime('%Y%m%d_%H%M%S')
            
            # Construye el nuevo nombre de archivo con la fecha actual
            new_filename = f"{filename[:-len(file_extension)]}_{current_datetime}{file_extension}"
            
            # Construye la ruta completa del archivo de origen
            source_filepath = os.path.join(source_directory, filename)

            # Construye la ruta completa del archivo de destino en la nueva ubicación
            target_filepath = os.path.join(target_directory, new_filename)

            # Mueve el archivo a la ubicación deseada
            shutil.move(source_filepath, target_filepath)

            if os.path.exists(target_filepath):
                print(f"Archivo movido correctamente a: {target_filepath}")
            else:
                print(f"No se pudo mover el archivo a la ubicación deseada: {target_filepath}")


source_directory = r'/home/bryan/Documentos/airflow/outputs'  # Directorio de descarga predeterminado
target_directory = r'/home/bryan/Documentos/airflow/outputs'  # Nueva ubicación de destino
file_extension = '.csv'  # Extensión del archivo a mover (ajusta según sea necesario)



gepard_automation("f.lozada@coperva.com", "12345678")
# Llama a la función para mover el archivo
move_downloaded_file(source_directory, target_directory, file_extension)

## Scraper marcatel

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os
from datetime import datetime, timedelta, date
from dotenv import load_dotenv, find_dotenv
import pandas as pd
import json
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import warnings
warnings.filterwarnings('ignore')

def marcatel_automation():
    url = "https://tink.marcatel.com.mx/Login.aspx"
    
    load_dotenv(find_dotenv()) # Load the .env file.
    userid = os.getenv("USER_MARCATEL")
    passwo = os.getenv("PASSWORD_MARCATEL")
    
   
    chrome_options = Options()
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(url)
    wait = WebDriverWait(driver, 10)
    user = wait.until(EC.presence_of_element_located((By.ID, "usuario")))
    user.click()
    user.send_keys(userid)
    contra = wait.until(EC.presence_of_element_located((By.ID, "contrasena")))
    
        # Pasar credenciales
    contra.click()
    contra.send_keys(passwo)
    init_button = wait.until(EC.element_to_be_clickable((By.ID, "btnini")))
    init_button.click()
    time.sleep(10)
        # Abrir ventana de reportes SMS
        # Redirecciona a la URL deseada
    driver.get("https://tink.marcatel.com.mx/Reportes/wfReporteRespuestas.aspx")
        # Espera unos segundos antes de cerrar el navegador
    time.sleep(10)
    
    # Get current date and time
    current_date = datetime.now()
    
    # Yesterday's date
    yersterday_date = current_date - timedelta(days=30)
    
    # Format date
    formatted_date = yersterday_date.strftime("%Y-%m-%d")
    print(formatted_date)
    # Date to marcatel
    date_marcatel = wait.until(EC.presence_of_element_located((By.ID, 'start')))
    date_marcatel.click()
    date_marcatel.clear()
    date_marcatel.send_keys(formatted_date)
    
    time.sleep(6)

    reporte = wait.until(EC.element_to_be_clickable((By.ID, "generarReporte")))
    reporte.click()
    time.sleep(30)
    exportar = driver.find_element(By.ID, "excel")
    driver.execute_script("arguments[0].scrollIntoView();", exportar)
    exportar.click()
    time.sleep(10)
    driver.quit()
marcatel_automation()


## Scraper Partidos Políticos y gobernadores

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import csv
import time
import re
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options
import os
import pandas as pd
import json
import warnings
warnings.filterwarnings('ignore')


def gobernadores_scraper():
    url = 'https://www.conago.org.mx/gobernadores'
    

   
    # Configuración para evitar notificaciones
    chrome_options = Options()
    #options.add_argument("window-size=1200x600")
    chrome_options.add_experimental_option("prefs", {
    "download.prompt_for_download": False,  # Desactiva la ventana emergente de descarga
    "download.directory_upgrade": True,
    "safebrowsing.enabled": False,  # Desactiva la verificación de seguridad de descargas
    #"profile.default_content_settings.popups":0,
    "download.default_directory":"/opt/airflow/outputs/Gobernadores/"})
    # Configuración para ingresar al explorador

    remote_webdriver = 'remote_chromedriver'
    driver = webdriver.Chrome( options=chrome_options)
    driver.get(url)

    # Localiza los elementos de interés
    elementos = driver.find_elements(By.CLASS_NAME, 'col-xs-12.col-md-8.texto.rcGobernadores')


    # Crear o abrir el archivo CSV
    with open('datos.csv', 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['Nombre', 'Fecha de Inicio', 'Fecha de Fin', 'Estado']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        writer.writeheader()  # Escribir la fila de encabezados
        
        for _ in range(len(elementos)):
            # Obtén el elemento específico y el estado
            elemento = elementos[_]
            estado_element = elemento.find_element(By.CLASS_NAME, 'textMedio')
            estado = estado_element.text
            estado_element.click()  # Hacer clic en el enlace
            
            # Obtener los elementos de interés en la página de detalle del estado
            elementos_estado = driver.find_elements(By.CLASS_NAME, 'col-xs-12.col-sm-6.col-md-4.pb15')

            for elemento_estado in elementos_estado:
                nombre_gobernador = elemento_estado.find_element(By.TAG_NAME, 'h4').text
                html = elemento_estado.get_attribute('innerHTML')
                
                # Usar expresión regular para encontrar las fechas
                fechas = re.search(r'(\d{2}/\d{2}/\d{4}) a (\d{2}/\d{2}/\d{4})', html)
                
                if fechas:
                    fecha_inicio = fechas.group(1)
                    fecha_fin = fechas.group(2)
                else:
                    fecha_inicio = "Fecha no disponible"
                    fecha_fin = "Fecha no disponible"
                
                # Imprimir para verificar
                print(nombre_gobernador, fecha_inicio, fecha_fin)
                
                # Escribir los datos en el archivo CSV
                writer.writerow({'Nombre': nombre_gobernador, 'Fecha de Inicio': fecha_inicio,
                                'Fecha de Fin': fecha_fin, 'Estado': estado})

            driver.back()  # Regresar a la página anterior

        # Cerrar el navegador
        driver.quit()
gobernadores_scraper()



def partidos_scraper():
    # Configura el controlador de Chrome
    driver = webdriver.Chrome()

    # Abre la página web de Wikipedia
    url = 'https://es.wikipedia.org/wiki/Wikipedia:Portada'
    driver.get(url)

    # Leer los nombres de los gobernadores desde el archivo CSV
    nombres_gobernadores = []
    resultados = []

    with open('datos.csv', 'r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            nombre_completo = row['Nombre']
            
            # Buscar el último punto en el nombre
            ultimo_punto_index = nombre_completo.rfind('.')
            if ultimo_punto_index != -1:
                nombre_completo = nombre_completo[ultimo_punto_index+1:].strip()
            
            nombres_gobernadores.append(nombre_completo)

    # Iterar sobre los nombres y buscar en la página web
    for nombre in nombres_gobernadores:
        buscar_gobernador = WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'cdx-text-input__input'))
        )

        # Usar ActionChains para borrar y escribir
        actions = ActionChains(driver)
        actions.click(buscar_gobernador).key_down(Keys.CONTROL).send_keys('a').key_up(Keys.CONTROL).send_keys(Keys.DELETE).send_keys(nombre).perform()
        buscar_gobernador = WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'cdx-text-input__input'))
        )
        time.sleep(5)
        buscar_gobernador.send_keys(Keys.ENTER)
        #button_search = WebDriverWait(driver, 10).until((EC.element_to_be_clickable((By.CLASS_NAME, 'cdx-button cdx-button--action-default cdx-button--weight-normal cdx-button--size-medium cdx-button--framed cdx-search-input__end-button'))))
        #button_search.click()
        #buscar_gobernador.submit()
        buscar_gobernador = WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'cdx-text-input__input'))
        )
        # Esperar a que se cargue la página del artículo
        driver.implicitly_wait(10)

        # Verificar si la página muestra resultados de búsqueda
        try:
            no_results_heading = driver.find_element(By.CLASS_NAME, 'firstHeading.mw-first-heading')
            if no_results_heading.text == 'Resultados de la búsqueda':
                print(f"No se encontró información para {nombre}")
                print("---")
                continue
        except NoSuchElementException:
            pass

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        infobox = soup.find('table', {'class': 'infobox biography vcard'})

        if infobox:
            partido_element = infobox.find('th', string='Partido político')
            if partido_element:
                partido_td = partido_element.find_next('td')
                partido = partido_td.get_text(strip=True)

                # Buscar el año dentro de los paréntesis
                anio_inicio_element = partido_td.find('span', {'style': 'font-size:85%;'})
                if anio_inicio_element:
                    anio_inicio = anio_inicio_element.get_text(strip=True).strip('()')
                else:
                    anio_inicio = "No disponible"

                resultados.append({"Nombre": nombre, "Partido político": partido, "Año de inicio": anio_inicio})
            else:
                resultados.append({"Nombre": nombre, "Partido político": "No disponible", "Año de inicio": "No disponible"})
        else:
            resultados.append({"Nombre": nombre, "Partido político": "No disponible", "Año de inicio": "No disponible"})

    # Guardar los resultados en un archivo CSV
    with open('resultados_gobernadores.csv', 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ["Nombre", "Partido político", "Año de inicio"]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(resultados)

    # Cerrar el navegador
    driver.quit()
partidos_scraper()

In [ ]:
df_gob = pd.read_csv("datos.csv")
df_gob['Nombre'] = df_gob['Nombre'].str.replace(r'\w+\.', '', regex=True)
df_gob['Nombre'] = df_gob['Nombre'].str.replace(r'\s', '', regex=True)
#df_gob = df_gob.drop_duplicates(subset=['Nombre'])
df_gob

In [ ]:
df_pp = pd.read_csv("resultados_gobernadores.csv")
df_pp['Nombre'] = df_pp['Nombre'].str.replace(r'\s', '', regex=True)
df_pp

In [ ]:
filtrado_ejemplo = df_pp.loc[df_pp['Año de inicio']!='No disponible']
filtrado_ejemplo

In [ ]:
df_merge = pd.merge(df_pp, df_gob, on='Nombre')

partidos = []
years = []

pattern = r'(\w+)\((\d{4})'
matches = df_merge['Partido político'].str.findall(pattern)

for match in matches:
    if match:
        m = match[0]
        partidos.append(m[0])
        year_partido = [int(year) for year in m[1].split('-')]
        year_max = max(year_partido)
        years.append(year_max)

data = {'Partido': partidos, 'Año': years}
df_temporal = pd.DataFrame(data)

df_merge = pd.concat([df_merge, df_temporal], axis=1)  # Fusionar con el DataFrame original

print(df_merge)



## Scraper SHF 

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import shutil
from datetime import datetime
from selenium.webdriver.common.action_chains import ActionChains
import requests
import PyPDF2
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

def ift_scraper():
    chrome_options = Options()
    #chrome_options.add_argument('--headless')
    #chrome_options.add_argument('--disable-dev-shm-usage')   
    chrome_options.add_experimental_option("prefs", {
    "download.prompt_for_download": False,  # Desactiva la ventana emergente de descarga
    "download.directory_upgrade": True,
    "safebrowsing.enabled": False,  # Desactiva la verificación de seguridad de descargas
    })
    
    #remote_webdriver = 'remote_chromedriver'
    driver = webdriver.Chrome( options=chrome_options)
    # Scraping part
    url = "https://www.gob.mx/shf/documentos/indice-shf-de-precios-de-la-vivienda-en-mexico-2021-a-2025?state=published"
    # Configuración para ingresar al explorador
    #driver = webdriver.Chrome(options = chrome_options)
    driver.get(url)
    wait= WebDriverWait(driver, 10)
        
        # Encuentra todos los elementos li con la clase "clearfix documents"
    document_elements = driver.find_elements(By.CSS_SELECTOR, "li.clearfix.documents")
        
        # Verifica si hay al menos un elemento
    if document_elements:
            # Encuentra el primer elemento de la lista
        first_document = document_elements[0]
            
            # Encuentra botón de descarga
        pdf_link = first_document.find_element(By.CSS_SELECTOR, "a[href$='.pdf']")
            
            # Obtiene el enlace del atributo href
        pdf_url = pdf_link.get_attribute("href")
            
            # Abre el enlace en una nueva ventana
        driver.execute_script("window.open('" + pdf_url + "', '_blanck');")
            
            # Cambia el enfoque a la nueva ventana
        driver.switch_to.window(driver.window_handles[-1])
            
            # Obtener URL actual
        url_actual = driver.current_url
        print("URL actual", url_actual)
            
            # Ruta destino
        ruta_destino = "SHF.pdf"
            
            # Realiza la solicitud HTTP para descargar el archivo
        response=requests.get(url_actual)
            
            # Verifica si la descarga fue exitosa (código de estado 200)
        if response.status_code == 200:
                # Abre un archivo en modo binario y guarda el contenido del PDF
            with open(ruta_destino, 'wb') as pdf_file:
                pdf_file.write(response.content)
    driver.quit()


    def pdf_convert():
        pdf_file = "SHF.pdf"
            
        # Abre el archivo PDF
        pdf_file = open(pdf_file, 'rb')

        # Crea un objeto PdfFileReader para leer el PDF
        pdf_reader = PyPDF2.PdfReader(pdf_file)

        # Inicializa una lista para almacenar el texto de cada página
        page_texts = []

        # Itera a través de las páginas y extrae el texto
        for page in pdf_reader.pages:
            page_texts.append(page.extract_text())

        # Cierra el archivo PDF
        pdf_file.close()

        # Combina el texto de todas las páginas en un solo texto
        all_text = '\n'.join(page_texts)

        # Divide el texto en líneas
        lines = all_text.split('\n')

        # Crea un DataFrame a partir de las líneas
        df = pd.DataFrame(lines, columns=['Texto'])

        ruta_csv = "SHF.csv"    
        df.to_csv(ruta_csv, index=False)
        return df
    pdf_convert()

    def extractor_tablas():
        df = pd.read_csv("SHF.csv")
        df = df.drop(df.index[48:])
        # Reajusta el índice para que comience desde 0
        df = df.reset_index(drop=True)
        # Elimina la primera fila, que contiene el encabezado, y guarda el resultado en un nuevo DataFrame
        df_sin_encabezado = df.iloc[1:].copy()

        # Extrae la columna 'Valores'
        df_sin_encabezado['Valores'] = df_sin_encabezado['Texto'].str.extract(r'(\d[\d. \s]+)')
        # Dividir la columna 'Valores' en múltiples columnas usando espacios en blanco como separadores
        columnas_valores = df_sin_encabezado['Valores'].str.split(expand=True)
        # Eliminar filas que contienen valores NaN (en este caso, la fila que contiene "Na")
        columnas_valores = columnas_valores.dropna()

        df_sin_encabezado['Estado'] = (df_sin_encabezado['Texto'].str.extract('([^0-9]+)'))
        # Combina la columna 'Estado' de df_sin_encabezado con columnas_valores
        columnas_valores.insert(0, 'Estado', df_sin_encabezado['Estado'].values)
        # Acceder a la segunda columna por posición
        segunda_columna = columnas_valores.iloc[:, 1]
        # Lista de índices de filas a modificar 
        filas_a_modificar = [0, 33, 34]

        for fila_idx in filas_a_modificar:
            # Modificar los valores de la segunda columna
            segunda_columna.iloc[fila_idx] = str(segunda_columna.iloc[fila_idx])[1:]

        columnas_valores = columnas_valores.rename(columns={columnas_valores.columns[0]: 'Estado'})


        #Obtener la longitud 
        num_columnas = len(columnas_valores.columns)
        # Crear una lista de años desde 2015 hasta 2024
        years = range(2015, 2025)
        # Crear una lista de números romanos del I al IV
        romanos = ['I', 'II', 'III', 'IV']
        # Crea una lista de nombres de columna basados en el patrón deseado
        nombres_columnas = ["Estado"] + [f"{anno}-{romano}" for anno in years for romano in romanos]
        # Asegurarse de que la lista de nombres de columna tenga la misma longitud que la tabla
        nombres_columnas = nombres_columnas[:num_columnas]
        columnas_valores.columns = nombres_columnas
        # Guardar el DataFrame modificado en un archivo CSV
        columnas_valores.to_csv("SHF_extract.csv", index=False)
    extractor_tablas()
ift_scraper()

## Scraper Desempleo

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import shutil
from datetime import datetime
from selenium.webdriver.common.action_chains import ActionChains
import requests
import PyPDF2
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import os



def INEGI_desempleo_scraper():

    # Get the directory of the current script
    script_directory = os.path.dirname(os.path.realpath(__file__))

    # Define the download directory relative to the script's location
    download_directory = os.path.join(script_directory, 'downloads')

    # Ensure the download directory exists
    if not os.path.exists(download_directory):
        os.makedirs(download_directory)

    url = "https://www.inegi.org.mx/app/tabulados/default.html?nc=624"
    
    chrome_options = Options()
    chrome_options.add_experimental_option("prefs", {
        "download.prompt_for_download": False,
        "download.default_directory": download_directory,  # Set to the dynamically determined path
        "download.directory_upgrade": True,
        "safebrowsing.enabled": True,
        "profile.default_content_settings.popups": 0,
        "profile.default_content_setting_values.automatic_downloads": 2,
    })

    # Configuración para ingresar al explorador
    #driver = webdriver.Chrome(options = chrome_options)
    remote_webdriver = 'remote_chromedriver'
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(url)
    wait= WebDriverWait(driver, 10)
    button = wait.until(EC.element_to_be_clickable((By.ID, "aCsv")))
    button.click()
    time.sleep(20)
    driver.quit()
    
    def data_transformation():
        df = pd.read_csv("Tabulado.csv")
        # Encuentra los índices donde comienzan los años en la columna 'Periodo'
        indices_anios = df[df['Periodo'].str.isnumeric()].index
        nuevo_df = df.T.iloc[1:].reset_index(drop=True)
        nuevo_df = nuevo_df.dropna(axis =1)
        num_columnas = len(nuevo_df.columns)
            
        years = range(2020,2025)
        # Lista de números romanos
        romanos = ['I', 'II', 'III', 'IV']
        # Crea una lista de nombres de columnas basados en el patrón
        nombres_columnas = [f"{anno}-{romano}" for anno in years for romano in romanos]
        nombres_columnas = nombres_columnas[:num_columnas]
        #Asigna los nombres de las columnas al nuevo DataFrame
        nuevo_df.columns = nombres_columnas 
        # Agrega una columna 'Estado' al nuevo DataFrame
        nuevo_df.insert(0, 'Estado', df.columns[1:])

        # Restablece el índice
        nuevo_df.reset_index(drop=True, inplace=True)
        nuevo_df.to_csv("Tabulado.csv", index = False)
    data_transformation()
INEGI_desempleo_scraper()

## Scraper Clima

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os
import pandas as pd
import json
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import warnings
from bs4 import BeautifulSoup
import json
import csv
from urllib.parse import urlparse
from selenium.common.exceptions import WebDriverException

warnings.filterwarnings('ignore')

def obtener_nombre_estado(url):
    parsed_url = urlparse(url)
    path = parsed_url.path
    parts = path.split("/")
    if len(parts) >= 3:
        estado = parts[2].replace("-", " ")
        # Tratar la excepción de "distrito-federal"
        if "estado-de" in estado:
            estado = estado.replace("estado-de-", "")
        elif "distrito-federal" in estado:
            estado = "Ciudad de México"
        return estado
    return None


def clima_scraper():
    max_attempts = 4
    attempts = 0

    while attempts < max_attempts:
        datos_climaticos = []
        try:

            urls =  [
            "https://www.clima.com/mexico/estado-de-aguascalientes/aguascalientes",
            "https://www.clima.com/mexico/estado-de-baja-california/mexicali",
            "https://www.clima.com/mexico/estado-de-baja-california-sur/la-paz",
            "https://www.clima.com/mexico/estado-de-campeche/campeche",
            "https://www.clima.com/mexico/estado-de-chiapas/tuxtla-gutierrez",
            "https://www.clima.com/mexico/estado-de-chihuahua/chihuahua",
            "https://www.clima.com/mexico/distrito-federal/mexico",
            "https://www.clima.com/mexico/estado-de-coahuila-de-zaragoza/saltillo",
            "https://www.clima.com/mexico/estado-de-colima/colima",
            "https://www.clima.com/mexico/estado-de-durango/victoria-de-durango",
            "https://www.clima.com/mexico/estado-de-guanajuato/guanajuato",
            "https://www.clima.com/mexico/estado-de-guerrero/chilpancingo-de-los-bravos",
            "https://www.clima.com/mexico/estado-de-hidalgo/pachuca-de-soto",
            "https://www.clima.com/mexico/estado-de-jalisco/guadalajara",
            "https://www.clima.com/mexico/estado-de-mexico/ecatepec",
            "https://www.clima.com/mexico/estado-de-michoacan-de-ocampo/morelia",
            "https://www.clima.com/mexico/estado-de-morelos/cuernavaca",
            "https://www.clima.com/mexico/estado-de-nayarit/tepic",
            "https://www.clima.com/mexico/estado-de-nuevo-leon/monterrey",
            "https://www.clima.com/mexico/estado-de-oaxaca/oaxaca-de-juarez",
            "https://www.clima.com/mexico/estado-de-puebla/puebla-de-zaragoza",
            "https://www.clima.com/mexico/estado-de-queretaro-de-arteaga/queretaro",
            "https://www.clima.com/mexico/estado-de-quintana-roo/cancun-2",
            "https://www.clima.com/mexico/estado-de-san-luis-potosi/san-luis-potosi",
            "https://www.clima.com/mexico/estado-de-sinaloa/culiacan",
            "https://www.clima.com/mexico/estado-de-sonora/hermosillo",
            "https://www.clima.com/mexico/estado-de-tabasco/villahermosa",
            "https://www.clima.com/mexico/estado-de-tamaulipas/reynosa",
            "https://www.clima.com/mexico/estado-de-tlaxcala/tlaxcala",
            "https://www.clima.com/mexico/estado-de-veracruz-llave/veracruz",
            "https://www.clima.com/mexico/estado-de-yucatan/merida",
            "https://www.clima.com/mexico/estado-de-zacatecas/zacatecas"
        ]
        
            
            chrome_options = Options()
            driver = webdriver.Chrome(options=chrome_options)

            for url in urls:
                driver.get(url)
                wait = WebDriverWait(driver, 10)

                container= wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "table")))
                container.location_once_scrolled_into_view

                desplazamiento_vertical = 400  # Cambia esto a la cantidad de píxeles que desees desplazarte

                # Realiza el desplazamiento
                driver.execute_script(f"window.scrollBy(0, {desplazamiento_vertical});")

                time.sleep(10)

                #driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                page_soruce = driver.page_source
                soup = BeautifulSoup(page_soruce, "html.parser")
                # Encontrar el elemento <canvas>
                canvas = soup.find('canvas', class_='chart gtm-climateAverage-chart')

                # Obtener el valor del atributo 'data-months'
                data_months = canvas['data-months']

                
                months_data = json.loads(data_months)
                # Obtener el nombre del estado
                nombre_estado = obtener_nombre_estado(url)

                for mes in months_data:
                    mes['Estado'] = nombre_estado # Agregar la nueva columna con el nombre del estado

                datos_climaticos.extend(months_data)

                # Crear un archivo CSV y escribir los datos
                with open('datos_climaticos.csv', mode='w', newline='') as file:
                    fieldnames = list(months_data[0].keys())  # Obtener los nombres de las columnas
                    #fieldnames.append('Estado')  # Agregar la nueva columna 'Estado'
                    writer = csv.DictWriter(file, fieldnames=fieldnames)
                    writer.writeheader()
                    writer.writerows(datos_climaticos)
            break
        except WebDriverException as e:
            print(f"Se produjo una excepción: {e}")
            attempts += 1
            print(f"Reintentando (Intento {attempts}/{max_attempts})...")
            driver.quit()
            continue
    if attempts == max_attempts:
        print(f"Se alcanzó el número máximo de intentos")
        driver.quit()

clima_scraper()

In [ ]:
df_gob = pd.read_csv("datos.csv")
df_gob['Nombre'] = df_gob['Nombre'].str.replace(r'\w+\.', '', regex=True)
df_gob['Nombre'] = df_gob['Nombre'].str.replace(r'\s', '', regex=True)
#df_gob = df_gob.drop_duplicates(subset=['Nombre'])
df_gob

## Union de datos

### Acomodo gobernador

In [ ]:
df_gob = pd.read_csv("datos.csv")
df_gob['Nombre'] = df_gob['Nombre'].str.replace(r'\w+\.', '', regex=True)
df_gob['Nombre'] = df_gob['Nombre'].str.replace(r'\s', '', regex=True)
df_gob['Estado'] = df_gob['Estado'].str.lower()
df_gob

### Acomodo por partido Político por gobernador

In [ ]:
df_pp = pd.read_csv("resultados_gobernadores.csv")
df_pp

In [ ]:
df_pp = pd.read_csv("resultados_gobernadores.csv")
df_pp['Nombre'] = df_pp['Nombre'].str.replace(r'\s', '', regex=True)

def split_parties(row):
    party = None
    year = None

    match = re.search(r'(.+?)\(([^)]+)\)', row)
    if match:
        party = match.group(1)
        years_in_match = re.findall(r'\d{4}', match.group(2))
        if years_in_match:
            year = max(int(year) for year in years_in_match)
    
    return party, year


# Aplicar la función a cada fila
split_data = df_pp['Partido político'].apply(split_parties)
df_pp['Partido'] = split_data.apply(lambda x: x[0])
df_pp['Año'] = split_data.apply(lambda x: x[1])
df_pp['Partido'].fillna(df_pp['Partido político'], inplace=True)

df_pp= df_pp[['Nombre', 'Partido']]
print(df_pp)


### Merge entre gobernadores y partidos políticos

In [ ]:
df_merge = pd.merge(df_pp, df_gob, on='Nombre')
df_merge


### Acomodo SHF

In [ ]:
df_shf = pd.read_csv('SHF_extract.csv')
df_shf = df_shf.drop(df_shf.index[33:])
df_shf = df_shf.melt(id_vars=['Estado'], var_name = 'Trimestre', value_name='SHF')
# Usar str.contains para filtrar los registros que contienen "2023"
df_shf = df_shf[df_shf['Trimestre'].str.contains('2023')]
df_shf['Estado'] = df_shf['Estado'].str.lower()
df_shf

### Acomodo desempleo

In [ ]:
from unidecode import unidecode
df_desemp = pd.read_csv('Tabulado.csv')
df_desemp = df_desemp.melt(id_vars=['Estado'], var_name='Trimestre', value_name='DP')

renombre = {
    'Coahuila de Zaragoza' : 'Coahuila',
    'Distrito Federal' : 'Ciudad de México',
    'Michoacán de Ocampo' : 'Michoacán',
    'Veracruz de Ignacio de la Llave':'Veracruz'
}

df_desemp['Estado']=df_desemp['Estado'].replace(renombre)
df_desemp['Estado'] = df_desemp['Estado'].str.lower() 
df_desemp['Estado'] = df_desemp['Estado'].apply(unidecode)
df_desemp


### Acomodo datos climáticos

In [ ]:
df_clima = pd.read_csv('datos_climaticos.csv')
df_clima['Estado'] = df_clima['Estado'].str.replace(r'^estado de ', '', regex=True)
renombre = {
    'coahuila de zaragoza' : 'coahuila',
    'distrito federal' : 'ciudad de mexico',
    'michoacan de ocampo' : 'michoacan',
    'veracruz llave':'veracruz',
    'queretaro de arteaga': 'queretaro',
}

df_clima['Estado']=df_clima['Estado'].replace(renombre)
df_clima


In [ ]:
# Mapeo de meses a trimestres
meses_a_trimestres = {
    'Enero': 'I',
    'Febrero': 'I',
    'Marzo': 'I',
    'Abril': 'II',
    'Mayo': 'II',
    'Junio': 'II',
    'Julio': 'III',
    'Agosto': 'III',
    'Septiembre': 'III',
    'Octubre': 'IV',
    'Noviembre': 'IV',
    'Diciembre': 'IV'
}

# Agregar una columna de Trimestre al DataFrame de meses
df_clima['Trimestre'] = df_clima['name'].map(meses_a_trimestres)

# Fusionar DataFrames en función del Estado y el Trimestre
df_final = df_clima.merge(df_desemp, on=['Estado', 'Trimestre'])

# Mostrar el DataFrame resultante
print(df_final)

## Scraper REPEP

In [ ]:
telefonos = ['5514602894', '5557780290', '3316234252']
def repep_scraper():
    repep = []
    for telefono in telefonos :
        chrome_options = Options()
        # Configuraciones adicionales omitidas para brevedad

        driver = webdriver.Chrome(options=chrome_options)
        url = "https://repep.profeco.gob.mx/Solicitudnumero.jsp"
        driver.get(url)
        wait = WebDriverWait(driver, 10)
            
        # Encuentra el elemento y realiza las acciones
        input_repep = driver.find_element(By.ID, "telefono")
        input_repep.click()  # Primero haz click en el elemento
        input_repep.send_keys(telefono)  # Luego envía las teclas
        input_repep.submit()  # Finalmente, realiza el envío del formulario
        time.sleep(5)
        try : 
            #Esperar por la vetnana emergente durante 10 minutos
            WebDriverWait(driver, 10).until(EC.alert_is_present())
            #Cambiar el enfoque a la vetnana emergente
            alert = driver.switch_to.alert
            # Obtener el texto de la ventana emergente
            alert_text = alert.text
            print(f'Alerta encontrada:{alert_text}')
            alert.accept()
        except TimeoutException:
            # Si no se encuentra ninguna ventana emergente en 10 segundos
            print("No se encontró la ventana emergente.")
            repep.append(telefono)
            print(repep)
    
        driver.quit()
    repep_df = pd.DataFrame(repep, columns=['Telefono'])
    repep_df.to_csv('repep.csv', index=False)
    
repep_scraper()



# Scrapers para Calidad


## Nuxiba 

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from dotenv import load_dotenv, find_dotenv
import pandas as pd
import shutil
from datetime import datetime
from selenium.webdriver.common.action_chains import ActionChains
import requests
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities


def nuxiba_calls():
    url = 'https://cwkolob182.nuxiba.com/adminkolob/?softphone=WebRTC#/'
    load_dotenv(find_dotenv())
    
    chrome_options = Options()
    chrome_options.add_experimental_option("prefs", {
    "download.prompt_for_download": False,  # Desactiva la ventana emergente de descarga
    "download.directory_upgrade": True,
    "safebrowsing.enabled": False,  # Desactiva la verificación de seguridad de descargas
    #"download.default_directory": "C:/Users/DSTHREE/Documents/GITHUB/airflow/dags/utils",
})
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(url)
    
    wait= WebDriverWait(driver, 20)
    userid = os.getenv("USER_NUXIBA")
    user_entry = wait.until(EC.presence_of_element_located((By.ID, "logintxtUsername")))
    user_entry.send_keys(userid)
    password = os.getenv("PASSWORD_NUXIBA")
    password_entry = driver.find_element(By.CLASS_NAME, "inputPassword.form-control")
    password_entry.send_keys(password)
    password_entry.send_keys(Keys.ENTER)
    llamadas = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="finder-sidebar"]/i')))
    llamadas.click()
    time.sleep(10)



    driver.execute_script("document.body.style.zoom='50%'")
    time.sleep(2)



    data = []
    previous_num_of_rows = 0
    while True:
        table_rows = driver.find_elements(By.CSS_SELECTOR, '.sbft-rows tr')

        if len(table_rows) == previous_num_of_rows:  # No hay más registros nuevos
            break
        

        for row in table_rows[previous_num_of_rows:]:
            # ... [Tu código para extraer información y añadir a la lista `data`]
            # 1. Extraer información de la fila y guardarla en el diccionario `info`
            fecha = row.find_element(By.XPATH, './td[5]').text
            id_val = row.find_element(By.XPATH, './td[6]').text
            campana = row.find_element(By.XPATH, './td[7]').text
            canal = row.find_element(By.XPATH, './td[8]').text
            nombre_usuario = row.find_element(By.XPATH, './td[9]').text
            nombre_agente = row.find_element(By.XPATH, './td[10]').text
            telefono = row.find_element(By.XPATH, './td[11]').text
            duracion = row.find_element(By.XPATH, './td[12]').text
            calificacion = row.find_element(By.XPATH, './td[13]').text

            info = {
                "Fecha": fecha,
                "ID": id_val,
                "Campaña": campana,
                "Canal": canal,
                "Nombre de Usuario": nombre_usuario,
                "Nombre de Agente": nombre_agente,
                "Teléfono": telefono,
                "Duración": duracion,
                "Calificación": calificacion
            }

            data.append(info)

            element = wait.until(EC.presence_of_element_located((By.ID, "gcl-icon.checkbox"))) 
            if element.is_displayed():
                print("El elemento está visible")
            else:
                print("El elemento no está visible")


            display = element.value_of_css_property("display")
            visibility = element.value_of_css_property("visibility")

            if display == "none":
                print("El elemento tiene 'display: none'")
            if visibility == "hidden":
                print("El elemento tiene 'visibility: hidden'")

            try:
                element.click()
            except Exception as e:
                clicked_element = driver.execute_script("return document.elementFromPoint(arguments[0], arguments[1]);", *element.location.values())
                print(f"En lugar de tu elemento, el elemento {clicked_element.tag_name} con la clase {clicked_element.get_attribute('class')} está recibiendo el clic.")


            #checkbox.click()

            # Espera explícita hasta que el botón esté interactuable
            download = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'gcl-btn.tertiary ')))
            download.click()
            

        # Actualiza el contador
        previous_num_of_rows = len(table_rows)

        # Desplázate un poco hacia abajo para obtener más registros (puedes ajustar este valor según tus necesidades)
        driver.execute_script("window.scrollBy(0, 100);")
        time.sleep(2)

    # Después del bucle, procesa la lista `data`
    df = pd.DataFrame(data)
    print(df)

nuxiba_calls()

## CCC1 

### Load Configuration

In [29]:
# Configuration Handling
def load_configuration():
    """
    Load configuration from environment variables or a .env file.
    """

    # Obtén el directorio de trabajo actual
    application_path = os.getcwd()

    # Construye la ruta al archivo .env
    dotenv_path = os.path.join(application_path, '.env')

    load_dotenv(dotenv_path)
    # Accessing variables
    userid = os.getenv('USER_CC1')
    password = os.getenv('PASSWORD_CC1')

    return userid, password

### SetUp

In [59]:
# WebDriver Setup
def setup_webdriver():
    """
    Initializes and returns a Selenium WebDriver with configured options.
    """
    base_path = r"/home/bryan/Documentos/airflow/outputs/ccc1"
    current_date = datetime.now().strftime("%Y-%m-%d")
    folder_path = os.path.join(base_path, current_date)

    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    chrome_options = Options()
    chrome_options.add_experimental_option("prefs", {
        "profile.default_content_setting_values.notifications": 1,
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "safebrowsing.enabled": False,
        "download.default_directory": folder_path,
        "safebrowsing.disable_download_protection": True,
        "profile.default_content_setting_values.automatic_downloads": 1,
    })
    driver = webdriver.Chrome(options=chrome_options)
    return driver, folder_path

### Login

In [67]:
# Login Procedure
def login(driver, url, userid, password):
    """
    Logs into the website using provided credentials.
    
    :param driver: Selenium WebDriver instance
    :param url: URL to navigate to for login
    :param userid: User ID for login
    :param password: Password for login
    """
    driver.get(url)
    user_entry = driver.find_element(By.ID, "username")
    user_entry.send_keys(userid)
    password_entry = driver.find_element(By.ID, "password")
    password_entry.send_keys(password)
    login_button = driver.find_element(By.ID, "btnSave")
    login_button.click()

### Función para desplazarse y clickear

In [68]:
def desplazar_y_clickear(driver, boton_id):
    """
    Desplaza hasta el final de la página y luego hace clic en un botón dado su ID.
    
    Parámetros:
    - driver: Instancia del navegador Selenium WebDriver.
    - boton_id: String con el ID del botón a clickear.
    """
    # Desplazarse hasta el final de la página
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # Esperar hasta que el botón sea clickeable
    btn_next = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, boton_id))
    )
    
    # Desplazarse al elemento (opcional si el elemento ya está en la vista)
    ActionChains(driver).move_to_element(btn_next).perform()
    
    # Hacer clic en el botón
    btn_next.click()

### Carga de campañas

In [69]:
def main(cartera):
    """
    Main workflow for scraping data, processing it, and sending an email.

    :param cartera: The specific 'cartera' to scrape data for
    """
    # Load configuration and capture returned values
    userid, password = load_configuration()
    driver, folder_path = setup_webdriver()
    login(driver, "https://app.ccc.uno/Loading", userid, password)
    driver.maximize_window()
    time.sleep(5)
    try:
        driver.switch_to.frame(1)
    except NoSuchElementException:
        print("The specified frame does not exist.")
        return
    except Exception as e:
        print(f"Unexpected error when switching to frame: {e}")
        return
    
    wait = WebDriverWait(driver, 30)
    
    
    list_cartera = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'select2-selection__rendered')))
    list_cartera.click()
    search_input = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "select2-search__field")))
    search_input.send_keys(cartera)
    driver.switch_to.active_element.send_keys(Keys.ENTER)
    time.sleep(5)
    nueva_campaña= wait.until(EC.presence_of_element_located((By.ID, "btnCreateCampaign")))
    nueva_campaña.click()
    time.sleep(5)
    predictivo = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "label[for='predictive']")))
    predictivo.click()

    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    btn_next = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "btnNext"))
    )

    # Desplazarse al elemento
    ActionChains(driver).move_to_element(btn_next).perform()

    # Intentar hacer clic nuevamente
    btn_next.click()


    #time.sleep(10)

    nombre_campaña = wait.until(EC.presence_of_element_located((By.ID, 'name')))
    #nombre_campaña.click()
    nombre_campaña.send_keys('nombre_ejemplo')

    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    btn_next = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "btnNext"))
    )

    # Desplazarse al elemento
    ActionChains(driver).move_to_element(btn_next).perform()

    # Intentar hacer clic nuevamente
    btn_next.click()

    carga = wait.until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[1]/div/div/section/div/div[3]/div/form/fieldset/div[2]/div[3]/div/label')))
    carga.click()

    btn_next = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "btnNext"))
    )

    # Desplazarse al elemento
    ActionChains(driver).move_to_element(btn_next).perform()

    # Intentar hacer clic nuevamente
    btn_next.click()

    carga = wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/div[1]/div/div/section/div/div[3]/div/form/fieldset/div[2]/div[2]/div/label')))
    carga.click()


    # La ruta absoluta del archivo que deseas subir
    ruta_del_archivo = "C:/Users/bryan/Documents/airflow/outputs/ccc1_campañas/17_PRED_TOTALPLAY_NUEVA_ASIGNACION_190324.xlsx"

    # Localizar el elemento input para la carga de archivos por su ID y enviarle la ruta del archivo
    input_carga = driver.find_element(By.ID, "csv")
    input_carga.send_keys(ruta_del_archivo)

    # Ahora que el archivo está seleccionado, puedes interactuar con el botón de cargar archivo, si es necesario
    btn_upload = driver.find_element(By.ID, "btnUpload")
    # Antes de hacer clic en el botón de cargar, asegúrate de que el sitio web actualiza su estado (de deshabilitado a habilitado) adecuadamente.
    # Esto puede requerir esperar a que el estado del botón cambie.
    btn_upload.click()

    time.sleep(5)

    # Selección en la lista desplegable de "# Teléfono primario:"
    select_number = Select(driver.find_element(By.ID, "number"))
    select_number.select_by_value("TELEFONO_PRINCIPAL")  # Reemplaza "SPEAKER" por el valor que deseas seleccionar

    # Selección en la lista desplegable de "Clave Común:"
    select_key = Select(driver.find_element(By.ID, "key"))
    select_key.select_by_value("CUENTA")  # Reemplaza "TEXT" por el valor que deseas seleccionar

    # Selección en la lista desplegable de "# Teléfono secundario:"
    # Nota: Este select permite selecciones múltiples
    # select_secondary = Select(driver.find_element(By.ID, "secondary"))
    # select_secondary.select_by_value("TIME")  # Reemplaza "TIME" por el valor que deseas seleccionar
    # Si necesitas seleccionar múltiples valores, repite la línea anterior con los diferentes valores

    # Selección en la lista desplegable de "URL Personalizado:"
    select_customurl = Select(driver.find_element(By.ID, "customurl"))
    select_customurl.select_by_value("URL_CRM")  # Reemplaza "SPEAKER" por el valor que deseas seleccionar

    # Localizar la casilla de verificación por su ID y hacer clic en ella.
    checkbox = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="csvOpts"]/div[2]/div[1]/div[2]/div')))
    #ActionChains(driver).move_to_element(checkbox).perform()
    checkbox.click()

    # Localizar el botón por su ID y hacer clic en él.
    unlock_button = wait.until(EC.element_to_be_clickable((By.ID, "btnLockUnlock")))
    unlock_button.click()

    btn_next = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "btnNext"))
    )

    # Desplazarse al elemento
    ActionChains(driver).move_to_element(btn_next).perform()

    # Intentar hacer clic nuevamente
    btn_next.click()

    select_group_button = wait.until(EC.element_to_be_clickable((By.ID, "btnSelectGroup")))
    select_group_button.click()
    

    try:
        add_button_for_hsbc = wait.until(EC.element_to_be_clickable((By.XPATH, "//td[contains(text(), 'HSBC')]/following-sibling::td/button[contains(@class, 'btn-success')]")))
        # Si el botón es encontrado, haz clic en él
        add_button_for_hsbc.click()
        time.sleep(5)
        print("Botón 'Agregar' para HSBC fue clickeado exitosamente.")
    except TimeoutException:
        print("No se encontró el botón 'Agregar' para HSBC dentro del tiempo esperado.")

    select_group = wait.until(EC.element_to_be_clickable((By.ID, "grab_multi_accept_group")))
    select_group.click()
            


    btn_next = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "btnNext"))
    )

    # Desplazarse al elemento
    ActionChains(driver).move_to_element(btn_next).perform()

    # Intentar hacer clic nuevamente
    btn_next.click()


    # Espera hasta que el elemento que desencadena el reloj/calendario sea clickeable y haz clic en él.
    trigger_element = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, ".tpicker .input-group-addon"))
    )
    trigger_element.click()

    time.sleep(10)
    # except TimeoutException:
    #     print("Timeout while trying to interact with the cartera selection.")
    #     return
    # except Exception as e:
    #     print(f"Unexpected error during cartera selection: {e}")
    #     return
    
        
    #     Assume additional steps for file handling and email sending here...
    #     driver.quit()
    # except Exception as e:
    #     print(f"Error al intentar cambiar al frame: {e}")

main(1940)

ElementClickInterceptedException: Message: element click intercepted: Element <button id="btnNext" type="button" class="btn custom-primary">...</button> is not clickable at point (1188, 526). Other element would receive the click: <div class="tiny right" style="position:fixed; width:100%; padding:0px; bottom:0px; background-color:black; color:white;">...</div>
  (Session info: chrome=123.0.6312.58)
Stacktrace:
	GetHandleVerifier [0x00007FF7E3207062+63090]
	(No symbol) [0x00007FF7E3172CB2]
	(No symbol) [0x00007FF7E300EC65]
	(No symbol) [0x00007FF7E305BB34]
	(No symbol) [0x00007FF7E3059954]
	(No symbol) [0x00007FF7E3057164]
	(No symbol) [0x00007FF7E3055EF9]
	(No symbol) [0x00007FF7E304A708]
	(No symbol) [0x00007FF7E3076FDA]
	(No symbol) [0x00007FF7E304A00A]
	(No symbol) [0x00007FF7E30771F0]
	(No symbol) [0x00007FF7E3093412]
	(No symbol) [0x00007FF7E3076D83]
	(No symbol) [0x00007FF7E30483A8]
	(No symbol) [0x00007FF7E3049441]
	GetHandleVerifier [0x00007FF7E36025CD+4238301]
	GetHandleVerifi

### Grabaciones

In [ ]:
# Data Scraping and Processing
def scrape_and_process_data(driver, cartera, folder_path):
    """
    Navigates through the website, scrapes data, and saves it to a CSV file.
    
    :param driver: Selenium WebDriver instance
    :param cartera: The specific 'cartera' to scrape data for
    """
    data = []
    try:
        driver.switch_to.frame(1)
    except NoSuchElementException:
        print("The specified frame does not exist.")
        return
    except Exception as e:
        print(f"Unexpected error when switching to frame: {e}")
        return
    
    wait = WebDriverWait(driver, 30)
    
    try:
        list_cartera = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'select2-selection__rendered')))
        list_cartera.click()
        search_input = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "select2-search__field")))
        search_input.send_keys(cartera)
        driver.switch_to.active_element.send_keys(Keys.ENTER)
        #time.sleep(10)
    except TimeoutException:
        print("Timeout while trying to interact with the cartera selection.")
        return
    except Exception as e:
        print(f"Unexpected error during cartera selection: {e}")
        return

    max_retries = 3
    attempts = 0

    while attempts < max_retries:
        try:
            driver.get("https://app.ccc.uno/CallRecord")
            grabaciones = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="QFiltersCallStatus-CallRecord"]/label[4]/span')))
            break  # If the page loads and element is found, exit the loop
        except WebDriverException as e:
            print(f"Attempt {attempts + 1} failed: {e}")
            attempts += 1
            if attempts == max_retries:
                print("Maximum retries reached. Exiting.")
                driver.quit()  # Quitting the driver as a last resort if all retries fail
                break
            print("Retrying...")
            time.sleep(5)  # Wait for 5 seconds before retrying
    else:
        print("Operation successful after", attempts, "attempts.")
    
    try:
        grabaciones.click()
        #time.sleep(30)  # Consider replacing with a more reliable waiting mechanism
    except TimeoutException:
        print("Timeout while trying to select grabaciones.")
        return

    time.sleep(5)
    #  Wait for the specific message to appear
    loading_message_locator = (By.ID, "divBackgridStatusText-CallRecord")
    expected_messages = ["Hay una Buscar en progreso, por favor espere hasta que se complete para empezar una nueva.",
                        "  Cargando ..."]

    # Attempt to locate the message element without waiting for a specific message
    try:
        message_element = driver.find_element(*loading_message_locator)
        current_message = message_element.text.strip()
        print("Message found: ", current_message)
        
        # If the specific loading message is present, then proceed with clicking the refresh button
        if current_message in expected_messages:
            print("Message indicating search in progress is present.")
            
            attempts = 0
            max_attempts = 5
            while attempts < max_attempts:
                try:
                    refresh_button = wait.until(EC.element_to_be_clickable((By.ID, "btnBackgridRefresh-CallRecord")))
                    refresh_button.click()
                    print("Refresh button clicked successfully.")
                    time.sleep(5)  # Consider replacing with more dynamic waiting

                    # Check for the message to change
                    new_message = driver.find_element(*loading_message_locator).text.strip()
                    if new_message not in expected_messages:
                        print("Message changed, indicating the page has loaded.")
                        break
                except WebDriverException as e:
                    print(f"Attempt {attempts + 1} failed: {e}")
                
                attempts += 1

            if attempts == max_attempts:
                print("Max attempts reached, moving on to data extraction.")
        else:
            print("No relevant loading message, proceeding with data extraction.")
            
    except NoSuchElementException:
        # If the message element isn't found at all, proceed with data extraction
        print("Loading message element not found, proceeding with data extraction.")
    except TimeoutException:
        # If waiting for the refresh button times out, handle accordingly
        print("Timeout while waiting for the refresh button, proceeding with caution.")

    while True:
        try: 
            wait.until(
                EC.presence_of_element_located((By.CSS_SELECTOR, ".backgrid-paginator"))
            )
            table = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'table-responsive')))
            # Obtener el contenido HTML de la tabla
            table_html = table.get_attribute("outerHTML")
            # Analizar la tabla con BeautifulSoup
            soup = BeautifulSoup(table_html, "html.parser")
            
            time.sleep(5)
            rows_ = wait.until(EC.presence_of_all_elements_located((By.TAG_NAME, 'tr')))
            for row in rows_:
                #driver.execute_script("document.body.style.zoom='50%'")
                buttons = row.find_elements(By.TAG_NAME, 'button')
                for button in buttons:
                    onclick = button.get_attribute('onclick')
                    if onclick and 'DownloadCallRecording' in onclick:
                        driver.execute_script("arguments[0].scrollIntoView();", button)
                        # Wait for button to be clickable
                        wait.until(EC.element_to_be_clickable((button)))
                        try:
                            button.click()
                        except ElementClickInterceptedException:
                            print("Button was not clickable.")
                            
                    soup = BeautifulSoup(driver.page_source, "html.parser")
                    rows = soup.find_all("tr")
                    
                    for row in rows:
                        cells = row.find_all("td")
                        row_data = [cell.text.strip() for cell in cells]
                        data.append(row_data) 
            if data:
                # Supongamos que las columnas de tu DataFrame tienen estos nombres
                column_names = ["ID", "Tipo", "Campaña", "Agente", "Origen", "Número", "Destino", "Estatus", "Duración", "Facturable", "Tarifa / Min", "Costo", "Botón1", "Botón2", "Inicio"]

                # Crear un DataFrame de Pandas
                df_pandas = pd.DataFrame(data, columns=column_names)

                # Nombre del archivo CSV (puedes cambiarlo según tus preferencias)
                csv_filename = 'datos_extraidos.csv'

                # Ruta completa del archivo CSV
                csv_filepath = os.path.join(folder_path, csv_filename)
                
                # Check if the CSV file already exists.
                if os.path.isfile(csv_filepath):
                    # Load the existing data, append the new data, and save
                    existing_df = pd.read_csv(csv_filepath)
                    combined_df = pd.concat([existing_df, df_pandas], ignore_index=True)
                    combined_df = combined_df.drop_duplicates()
                else:
                    # If the file doesn't exist, the new data is the comdined data
                    combined_df = df_pandas.drop_duplicates()
                # Guarda el DataFrame combinado en un archivo CSV
                combined_df.to_csv(csv_filepath, index=False)


                pass
            # Check if 'Next' button is not disabled, then click it, otherwise break the loop
            next_button = driver.find_elements(By.XPATH,"//li[not(contains(@class, 'disabled'))]/a[@title='Siguiente']")
            if next_button:
                next_button[0].click()
                time.sleep(2)  # Adjust based on your page's loading time
            else:
                break  # Exit the loop if no more pages to navigate      
                

        except TimeoutException:
            print("Timeout occurred while trying to process a page.")
            break
        except Exception as e:
            print(f"Unexpected error during page processing: {e}")
            break

    # Final steps, like closing the driver, can be here
    driver.quit()

def navigate_to_next_page(driver):
    """
    Attempts to navigate to the next page and returns True if successful, False otherwise.
    
    :param driver: Selenium WebDriver instance
    :param wait: WebDriverWait instance for handling waits
    """
    try: 
        next_button = driver.find_element(By.XPATH, '//a[@title="Siguiente"]/i[@class="fa fa-angle-right fa-lg"]')
        # Verificar si el botón está deshabilitado
        if "disabled" in next_button.find_element(By.XPATH, ('../..')).get_attribute("class"):
            return False # Next button is disabled, no more pages to process
        next_button.click()
        time.sleep(5)
    except NoSuchElementException:
        print("Next button not found, possibly at the last page.")
        return False
    except Exception as e:
        print(f"Unexpected error when navigating to the next page: {e}")
        return False

# File Management
def rename_downloaded_file(folder_path):
    """
    Removes files with a number in parentheses in their name, then renames audio files to keep only the last number.
    
    :param folder_path: The directory where the audio files are stored.
    """
    pattern = re.compile(r'\(\d+\)')  # Regular expression to match number in parentheses
    for filename in os.listdir(folder_path):
        full_path = os.path.join(folder_path, filename)

        # Check if the filename matches the pattern for removal first
        if  filename.endswith(".crdownload") or pattern.search(filename):
            os.remove(full_path)
            print(f"Removed file '{filename}'")
            continue  # Skip to the next file
        
        # Then check if it's an audio file that needs renaming
        try:
            if '-' in filename and filename.endswith(".wav"):
                new_name = filename.split('-')[-1]  # Extract the last number from the file name
                new_file = os.path.join(folder_path, new_name)
                
                # Check if the new file name already exists
                if os.path.exists(new_file):
                    os.remove(new_file)  # Delete the existing file
                    print(f"Deleted existing file '{new_name}' to make way for renaming")
                
                os.rename(full_path, new_file)
                print(f"Renamed '{filename}' to '{new_name}'")
        except Exception as e:
            print(f"Error processing file '{filename}': {e}")

# def send_email(sender_email, receiver_email, subject, body, attachments, password):
#     """
#     Sends an email with attachments.
    
#     :param sender_email: Email address of the sender
#     :param receiver_email: Email address of the receiver
#     :param subject: Subject of the email
#     :param body: Body text of the email
#     :param attachments: List of file paths to attach
#     :param password: Password for the sender's email account
#     """
#     msg = MIMEMultipart()
#     msg['From'] = sender_email
#     msg['To'] = receiver_email
#     msg['Subject'] = subject
#     msg.attach(MIMEText(body, 'plain'))

#     for attachment in attachments:
#         part = MIMEBase('application', 'octet-stream')
#         with open(attachment, 'rb') as file:
#             part.set_payload(file.read())
#         encoders.encode_base64(part)
#         part.add_header('Content-Disposition', f"attachment; filename= {os.path.basename(attachment)}")
#         msg.attach(part)

#     server = smtplib.SMTP('smtp.gmail.com', 587)
#     server.starttls()
#     server.login(sender_email, password)
#     server.sendmail(sender_email, receiver_email, msg.as_string())
#     server.quit()

# Main Workflow
def main(cartera):
    """
    Main workflow for scraping data, processing it, and sending an email.

    :param cartera: The specific 'cartera' to scrape data for
    """
    # Load configuration and capture returned values
    userid, password = load_configuration()
    driver, folder_path = setup_webdriver()
    login(driver, "https://app.ccc.uno/Campaign", userid, password)
    scrape_and_process_data(driver, cartera, folder_path)
    time.sleep(5)
    rename_downloaded_file(folder_path)
    # Assume additional steps for file handling and email sending here...
    driver.quit()

if __name__ == "__main__":
    # The calls to main and other functions are commented out to prevent execution in this environment
    main(2803)

Unexpected error during cartera selection: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=123.0.6312.58)
Stacktrace:
	GetHandleVerifier [0x00007FF7E3207062+63090]
	(No symbol) [0x00007FF7E3172CB2]
	(No symbol) [0x00007FF7E300EC65]
	(No symbol) [0x00007FF7E2FECA7C]
	(No symbol) [0x00007FF7E307D687]
	(No symbol) [0x00007FF7E3092AC1]
	(No symbol) [0x00007FF7E3076D83]
	(No symbol) [0x00007FF7E30483A8]
	(No symbol) [0x00007FF7E3049441]
	GetHandleVerifier [0x00007FF7E36025CD+4238301]
	GetHandleVerifier [0x00007FF7E363F72D+4488509]
	GetHandleVerifier [0x00007FF7E3637A0F+4456479]
	GetHandleVerifier [0x00007FF7E32E05A6+953270]
	(No symbol) [0x00007FF7E317E57F]
	(No symbol) [0x00007FF7E3179254]
	(No symbol) [0x00007FF7E317938B]
	(No symbol) [0x00007FF7E3169BC4]
	BaseThreadInitThunk [0x00007FFA71114CB0+16]
	RtlUserThreadStart [0x00007FFA71B7E8AB+43]



NameError: name 're' is not defined

# Scrapper autocompara.com

## SetUp settings

In [ ]:
chrome_options = Options()
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_experimental_option("prefs", {
    "profile.default_content_setting_values.notifications": 1,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": False,
    #"download.default_directory": folder_path,
    "safebrowsing.disable_download_protection": True,
    "profile.default_content_setting_values.automatic_downloads": 1,
})

driver = webdriver.Chrome(options=chrome_options)
driver.maximize_window()

## Login

In [ ]:
driver.get("https://www.autocompara.com/")

In [ ]:
# Suponiendo que ya has navegado a la página en cuestión
max_ciclos = 2  # Número total de ciclos de intentos (1 ciclo = 3 intentos + refresco)
ciclo_actual = 0
seleccionado = False
año_deseado = 2024

while not seleccionado and ciclo_actual < max_ciclos:
    intentos = 0
    max_intentos = 3  # Número máximo de intentos por ciclo

    while not seleccionado and intentos < max_intentos:
        try:
            elemento = driver.find_element(By.ID, "moto")
            driver.execute_script("arguments[0].click();", elemento)
            time.sleep(10)

            elemento_para_click = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "ng-arrow-wrapper"))
            )
            elemento_para_click.click()

            opcion_año = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, f"//span[@class='ng-option-label' and contains(text(), '{año_deseado}')]"))
            )
            opcion_año.click()


            campo_busqueda = WebDriverWait(driver, 10).until(
                EC.visibility_of_element_located((By.XPATH, '//*[@id="search"]/div/div'))
            )
            campo_busqueda.click()
            campo_busqueda.send_keys("Thriller 200")
            campo_busqueda.send_keys(Keys.RETURN)


            seleccionado = True
            print("Elemento seleccionado con éxito.")
            
        except (NoSuchElementException, TimeoutException, NoSuchWindowException) as e:
            print("Elemento no encontrado o no disponible. Intentando nuevamente...", str(e))
            intentos += 1  
            if intentos >= max_intentos:
                print("Máximo de intentos alcanzado para este ciclo. Refrescando la página...")
                driver.refresh()
                time.sleep(5)  # Pausa después de refrescar para dar tiempo a la página de cargar

    ciclo_actual += 1

if not seleccionado:
    print("No fue posible seleccionar el elemento después de los intentos permitidos.")


ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=122.0.6261.95)
Stacktrace:
	GetHandleVerifier [0x00007FF7A15AAD02+56930]
	(No symbol) [0x00007FF7A151F602]
	(No symbol) [0x00007FF7A13D419D]
	(No symbol) [0x00007FF7A14149E7]
	(No symbol) [0x00007FF7A1413662]
	(No symbol) [0x00007FF7A143BC9A]
	(No symbol) [0x00007FF7A140F09A]
	(No symbol) [0x00007FF7A143BEB0]
	(No symbol) [0x00007FF7A14581E2]
	(No symbol) [0x00007FF7A143BA43]
	(No symbol) [0x00007FF7A140D438]
	(No symbol) [0x00007FF7A140E4D1]
	GetHandleVerifier [0x00007FF7A1926F8D+3711213]
	GetHandleVerifier [0x00007FF7A19804CD+4077101]
	GetHandleVerifier [0x00007FF7A197865F+4044735]
	GetHandleVerifier [0x00007FF7A1649736+706710]
	(No symbol) [0x00007FF7A152B8DF]
	(No symbol) [0x00007FF7A1526AC4]
	(No symbol) [0x00007FF7A1526C1C]
	(No symbol) [0x00007FF7A15168D4]
	BaseThreadInitThunk [0x00007FFC7A6E4CB0+16]
	RtlUserThreadStart [0x00007FFC7B1CE8AB+43]
